In [ ]:
import os
import torch
import logging
import asyncio
from typing import List, Dict, Union, Optional

import torch
import torch.nn as nn
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig, 
    get_peft_model, 
    prepare_model_for_kbit_training,
    PeftModel,
    PeftConfig
)
from datasets import Dataset, load_dataset

class LocalLLMFineTuner:
    """
    Comprehensive fine-tuning manager for local language models
    Supports multiple fine-tuning strategies:
    1. Full parameter fine-tuning
    2. LoRA (Low-Rank Adaptation)
    3. Quantized fine-tuning
    """
    
    def __init__(
        self, 
        model_name: str = "microsoft/Phi-4-mini-instruct", 
        device: Optional[str] = None,
        logging_dir: str = "./logs"
    ):
        """
        Initialize the fine-tuning manager
        
        Args:
            model_name: HuggingFace model identifier
            device: Device to run the model on (None for auto-detection)
            logging_dir: Directory for storing training logs
        """
        self.model_name = model_name
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.logging_dir = logging_dir
        
        # Setup logging
        os.makedirs(logging_dir, exist_ok=True)
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(os.path.join(logging_dir, "finetuning.log")),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
        
        # Model and tokenizer placeholders
        self.original_model = None
        self.tokenizer = None
        self.fine_tuned_model = None
    
    def load_model(self, quantize: bool = False):
        """
        Load the base model and tokenizer
        
        Args:
            quantize: Whether to load the model in a quantized format
        """
        self.logger.info(f"Loading model {self.model_name}")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Load model
        load_kwargs = {
            "torch_dtype": torch.float16 if self.device == "cuda" else torch.float32,
            "device_map": self.device
        }
        
        if quantize:
            load_kwargs.update({
                "load_in_4bit": True,
                "quantization_config": BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_compute_dtype=torch.float16,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_use_double_quant=True
                )
            })
        
        self.original_model = AutoModelForCausalLM.from_pretrained(
            self.model_name, 
            **load_kwargs
        )
        
        self.logger.info("Model loaded successfully")
    
    def prepare_dataset(
        self, 
        train_data: Union[str, List[Dict], Dataset], 
        text_column: str = "text"
    ) -> Dataset:
        """
        Prepare dataset for fine-tuning
        
        Args:
            train_data: Training data source (file path, list of dicts, or existing dataset)
            text_column: Column name containing text data
        
        Returns:
            Processed dataset
        """
        # Handle different input types
        if isinstance(train_data, str):
            # Attempt to load from file or HuggingFace dataset
            try:
                dataset = load_dataset(train_data)
            except:
                # Assume it's a local file path
                dataset = load_dataset("text", data_files=train_data)
        elif isinstance(train_data, list):
            dataset = Dataset.from_list(train_data)
        elif isinstance(train_data, Dataset):
            dataset = train_data
        else:
            raise ValueError("Unsupported dataset type")
        
        # Tokenize dataset
        def tokenize_function(examples):
            return self.tokenizer(
                examples[text_column], 
                truncation=True, 
                padding="max_length", 
                max_length=512
            )
        
        tokenized_dataset = dataset.map(
            tokenize_function, 
            batched=True, 
            remove_columns=dataset.column_names
        )
        
        return tokenized_dataset
    
    def setup_lora(
        self, 
        r: int = 16, 
        lora_alpha: int = 32, 
        lora_dropout: float = 0.1
    ):
        """
        Setup LoRA configuration for efficient fine-tuning
        
        Args:
            r: Rank of the low-rank adaptation
            lora_alpha: Scaling factor for LoRA
            lora_dropout: Dropout probability
        """
        if not self.original_model:
            self.load_model()
        
        # Prepare model for LoRA training
        self.original_model = prepare_model_for_kbit_training(self.original_model)
        
        # LoRA configuration
        lora_config = LoraConfig(
            r=r,
            lora_alpha=lora_alpha,
            target_modules=["q_proj", "v_proj"],
            lora_dropout=lora_dropout,
            bias="none",
            task_type="CAUSAL_LM"
        )
        
        # Create LoRA model
        self.fine_tuned_model = get_peft_model(self.original_model, lora_config)
        
        self.logger.info("LoRA configuration completed")
    
    def fine_tune(
        self, 
        train_data: Union[str, List[Dict], Dataset],
        method: str = "lora",
        epochs: int = 3,
        batch_size: int = 4,
        learning_rate: float = 2e-5
    ):
        """
        Fine-tune the model using specified method
        
        Args:
            train_data: Training data
            method: Fine-tuning method ('lora', 'full', 'quantized')
            epochs: Number of training epochs
            batch_size: Training batch size
            learning_rate: Learning rate for training
        """
        # Prepare dataset
        tokenized_dataset = self.prepare_dataset(train_data)
        
        # Split dataset
        train_dataset = tokenized_dataset.shuffle(seed=42).select(range(int(len(tokenized_dataset)*0.8)))
        eval_dataset = tokenized_dataset.shuffle(seed=42).select(range(int(len(tokenized_dataset)*0.2)))
        
        # Prepare model based on method
        if method == "lora":
            self.setup_lora()
            model_to_train = self.fine_tuned_model
        elif method == "quantized":
            self.load_model(quantize=True)
            model_to_train = self.original_model
        else:  # Full fine-tuning
            if not self.original_model:
                self.load_model()
            model_to_train = self.original_model
        
        # Training arguments
        training_args = TrainingArguments(
            output_dir=os.path.join(self.logging_dir, "checkpoints"),
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            learning_rate=learning_rate,
            warmup_steps=100,
            logging_dir=self.logging_dir,
            logging_steps=10,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True
        )
        
        # Data collator
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer, 
            mlm=False
        )
        
        # Trainer
        trainer = Trainer(
            model=model_to_train,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            data_collator=data_collator
        )
        
        # Train
        trainer.train()
        
        # Save model
        if method == "lora":
            self.fine_tuned_model.save_pretrained(
                os.path.join(self.logging_dir, "lora_model")
            )
        else:
            model_to_train.save_pretrained(
                os.path.join(self.logging_dir, "fine_tuned_model")
            )
        
        self.logger.info(f"Fine-tuning completed using {method} method")
    
    def log_training_metrics(self, trainer):
        """
        Log training metrics and performance
        
        Args:
            trainer: Trainer object from fine-tuning process
        """
        # Extract and log key metrics
        metrics = trainer.evaluate()
        loss = metrics.get('eval_loss', None)
        perplexity = math.exp(loss) if loss else None
        
        log_data = {
            "method": "lora",  # or other method
            "loss": loss,
            "perplexity": perplexity,
            "training_time": metrics.get('train_runtime', None),
            "samples_per_second": metrics.get('train_samples_per_second', None)
        }
        
        # Write to log file
        with open(os.path.join(self.logging_dir, "training_metrics.json"), "w") as f:
            json.dump(log_data, f, indent=4)
        
        self.logger.info("Training metrics logged successfully")
    
    def load_fine_tuned_model(self, model_path: str, method: str = "lora"):
        """
        Load a previously fine-tuned model
        
        Args:
            model_path: Path to the fine-tuned model
            method: Fine-tuning method used
        """
        if not self.original_model:
            self.load_model()
        
        if method == "lora":
            # Load LoRA model
            self.fine_tuned_model = PeftModel.from_pretrained(
                self.original_model, 
                model_path
            )
        else:
            # Load full fine-tuned model
            self.fine_tuned_model = AutoModelForCausalLM.from_pretrained(
                model_path,
                device_map=self.device
            )
        
        self.logger.info(f"Fine-tuned model loaded from {model_path}")

# Example usage
if __name__ == "__main__":
    # Initialize fine-tuner
    fine_tuner = LocalLLMFineTuner(
        model_name="microsoft/Phi-4-mini-instruct"
    )
    
    # Prepare your training data
    training_data = [
        {"text": "Your first training example"},
        {"text": "Another training example"}
    ]
    
    # Fine-tune using LoRA
    fine_tuner.fine_tune(
        train_data=training_data, 
        method="lora", 
        epochs=3
    )
    
    # Alternatively, full fine-tuning
    fine_tuner.fine_tune(
        train_data=training_data, 
        method="full", 
        epochs=3
    )